# Variables definition

In [1]:
openai_endpoint       = "https://mmopenaiscus.openai.azure.com/"
openai_api_key        = "3365c19fc5374b469e926fcd40777eee"
openai_api_version    = "2024-05-01-preview"
azure_deployment_name = "gpt-4o-for-apim"

apim_endpoint         = "https://mmapim02.azure-api.net/balanced-azure-openai-service-api/"
apim_subscription_key = "afb487599b0546d0a1b01e7474982cff"
apim_api_version      = "2023-09-01-preview" # never used

messages = [
    {
      "role": "user",
      "content": "What is a meaning function? Please summarize in less than 20 words."
    }
]

# Invoking Open AI via HTTP

In [2]:
import requests

messages_json = {"messages": messages}

url = f"{openai_endpoint}openai/deployments/{azure_deployment_name}/chat/completions?api-version={openai_api_version}"
print(f'This is the Open AI endpoint and body that is invoked:\n- {url}\n- {messages_json}')

This is the Open AI endpoint and body that is invoked:
- https://mmopenaiscus.openai.azure.com/openai/deployments/gpt-4o-for-apim/chat/completions?api-version=2024-05-01-preview
- {'messages': [{'role': 'user', 'content': 'What is a meaning function? Please summarize in less than 20 words.'}]}


In [3]:
response_http = requests.post(url, headers = {'api-key':openai_api_key}, json = {"messages": messages})

if (int(response_http.status_code / 100))==2:
    print(f"{response_http.json()['choices'][0]['message']['content']}\nx-ms-region = {response_http.headers['x-ms-region']}")
else:
    print(response_http.text)

A meaning function assigns semantic values to expressions in a language, determining their meaning.
x-ms-region = South Central US


# Invoking Open AI via Azure SDK

In [4]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint = openai_endpoint,
    api_key        = openai_api_key,
    api_version    = openai_api_version
)

In [5]:
response_sdk = client.chat.completions.create(
    model    = azure_deployment_name,
    messages = messages)

print(response_sdk.choices[0].message.content)

A meaning function links linguistic expressions to their semantic interpretations within a given context or language system.


# Invoking APIM via HTTP

In [12]:
import requests

messages_json = {"messages": messages}

url = f"{apim_endpoint}openai/deployments/{azure_deployment_name}/chat/completions?api-version={openai_api_version}"
print(f'This is the Open AI endpoint and body that is invoked:\n- {url}\n- {messages_json}')

This is the Open AI endpoint and body that is invoked:
- https://mmapim02.azure-api.net/balanced-azure-openai-service-api/openai/deployments/gpt-4o-for-apim/chat/completions?api-version=2024-05-01-preview
- {'messages': [{'role': 'user', 'content': 'What is a meaning function? Please summarize in less than 20 words.'}]}


In [13]:
response_http = requests.post(url, headers = {'api-key':apim_subscription_key}, json = {"messages": messages})
if (int(response_http.status_code / 100))==2:
    print(f"{response_http.json()['choices'][0]['message']['content']}\nx-ms-region = {response_http.headers['x-ms-region']}")
else:
    print(response_http.text)

A meaning function maps linguistic expressions to their semantic interpretations in formal semantics.
x-ms-region = Sweden Central


# Invoking APIM via Azure SDK

In [14]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint = apim_endpoint,
    api_key        = apim_subscription_key,
    api_version    = openai_api_version
)

In [15]:
response_sdk = client.chat.completions.create(
    model    = azure_deployment_name,
    messages = messages)

print(response_sdk.choices[0].message.content)

A meaning function assigns semantic values or interpretations to linguistic expressions within a formal system or logic framework.


## Loading test

In [16]:
from datetime import datetime
for i in range(200):
    response_sdk = client.chat.completions.create(
        model    = azure_deployment_name,
        messages = messages)

    print(f"step {i} at {datetime.now()}: {response_sdk.choices[0].message.content}")

step 0 at 2024-06-29 21:56:10.153877: A meaning function maps linguistic expressions to their interpretations, representing how words and sentences convey meaning.
step 1 at 2024-06-29 21:56:20.803325: A meaning function assigns interpreted meanings or semantic values to linguistic expressions or symbols.
step 2 at 2024-06-29 21:56:21.214829: A meaning function assigns semantic values to expressions within a language, defining their meanings systematically.
step 3 at 2024-06-29 21:56:21.620944: A meaning function assigns semantic values to linguistic expressions within a formal system or computational model.
step 4 at 2024-06-29 21:56:23.087901: A meaning function assigns semantic values to expressions within a formal language or system, interpreting their significance.
step 5 at 2024-06-29 21:56:23.510641: A meaning function maps expressions in a language to their meanings or interpretations within a specific context.
step 6 at 2024-06-29 21:56:24.009650: A meaning function assigns va